# NLP: Сантимент анализ.


Ваша задача - определить тональность текста по шкале от 1 (негативный) до 10 (позитивный).  

Ввод: тексты, разделенные переводом строки.  
Вывод: для каждого текста из входных данных вывести тональную оценку от 1 до 10.  

Разделитель между выводами для разных текстов -- перевод строки.

In [1]:
import torch
from datasets import Dataset, DatasetDict
from torch.optim import AdamW
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Загрузка данных

### Загрузка первого датасета: IMDB 1-10

In [3]:
! pip install opendatasets --quiet

In [4]:
import opendatasets as od

In [5]:
od.download("https://www.kaggle.com/datasets/nisargchodavadiya/imdb-movie-reviews-with-ratings-50k")

Dataset URL: https://www.kaggle.com/datasets/nisargchodavadiya/imdb-movie-reviews-with-ratings-50k


100%|██████████| 51.7M/51.7M [00:00<00:00, 353MB/s]

In [6]:
df1=pd.read_csv("/content/imdb-movie-reviews-with-ratings-50k/imdb_sup.csv")
df1.head()

,Review,Rating,Sentiment
0,"Kurt Russell's chameleon-like performance, cou...",10,1
1,It was extremely low budget(it some scenes it ...,8,1
2,James Cagney is best known for his tough chara...,8,1
3,"Following the brilliant ""Goyôkiba"" (aka. ""Hanz...",8,1
4,One of the last classics of the French New Wav...,10,1


### Загрузка второго датасета: сообщения в социальных сетях 1-3

In [7]:
od.download("https://www.kaggle.com/datasets/mdismielhossenabir/sentiment-analysis")

Dataset URL: https://www.kaggle.com/datasets/mdismielhossenabir/sentiment-analysis


100%|██████████| 14.3k/14.3k [00:00<00:00, 48.7MB/s]

In [8]:
df2=pd.read_csv("/content/sentiment-analysis/sentiment_analysis.csv")
df2.head()

,Year,Month,Day,Time of Tweet,text,sentiment,Platform
0,2018,8,18,morning,What a great day!!! Looks like dream.,positive,Twitter
1,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,Facebook
2,2017,8,18,night,Don't angry me,negative,Facebook
3,2022,6,8,morning,We attend in the class just for listening teac...,negative,Facebook
4,2022,6,8,noon,"Those who want to go, let them go",negative,Instagram


In [9]:
label_map = {
    "negative": 1.0,
    "neutral": 5.0,
    "positive": 10.0
}

df2["sentiment"] = df2["sentiment"].map(label_map)

#### Объединение датасетов

In [10]:
combined_texts = list(df1['Review']) + list(df2['text'])
combined_labels = list(df1['Rating']) + list(df2['sentiment'])

## Предобработка данных/токенайзер

In [11]:
train_texts, test_val_texts, train_labels, test_val_labels = train_test_split(
    combined_texts, combined_labels, test_size=0.3, shuffle=True, random_state=42
)

test_texts, val_texts, test_labels, val_labels = train_test_split(
    test_val_texts, test_val_labels, test_size=0.5, shuffle=True, random_state=42
)

In [ ]:
print(f"Train size: {len(train_texts)}")
print(f"Validation size: {len(val_texts)}")
print(f"Test size: {len(test_texts)}")

Train size: 35349
Validation size: 7575
Test size: 7575


In [12]:
train_dataset = Dataset.from_dict({
    "text": train_texts,
    "label": train_labels
})

val_dataset = Dataset.from_dict({
    "text": val_texts,
    "label": val_labels
})

test_dataset = Dataset.from_dict({
    "text": test_texts,
    "label": test_labels
})

dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [13]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

In [15]:
dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/35349 [00:00<?, ? examples/s]

Map:   0%|          | 0/7575 [00:00<?, ? examples/s]

Map:   0%|          | 0/7575 [00:00<?, ? examples/s]

## Загрузка модели

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=1,
    problem_type="regression"
)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Вычисление MAE

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    mae = mean_absolute_error(labels, preds)

    return {
        "mae": mae
    }

### Обучение

In [17]:
training_args = TrainingArguments(
    output_dir="./bert-sentiment-split",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="mae",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
)

In [24]:
trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    args=training_args
)

/tmp/ipython-input-2175837943.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

/tmp/ipython-input-2293080170.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Mae
1,3.324600,3.721487,5.618086
2,2.243700,3.658130,5.618086
3,1.655000,3.527301,5.618086


TrainOutput(global_step=6630, training_loss=2.9925616832520268, metrics={'train_runtime': 2646.3584, 'train_samples_per_second': 40.073, 'train_steps_per_second': 2.505, 'total_flos': 6975471891430656.0, 'train_loss': 2.9925616832520268, 'epoch': 3.0})

### Сохранение модели

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
!zip -r checkpoint.zip /content/content/bert-sentiment-split/checkpoint-6630

  adding: content/content/bert-sentiment-split/checkpoint-6630/ (stored 0%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/scheduler.pt (deflated 61%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/tokenizer.json (deflated 71%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/tokenizer_config.json (deflated 75%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/vocab.txt (deflated 53%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/rng_state.pth (deflated 26%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/optimizer.pt (deflated 13%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/trainer_state.json (deflated 80%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/special_tokens_map.json (deflated 42%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/config.json (deflated 49%)
  adding: content/content/bert-sentiment-split/checkpoint-6630/training_args.bin

In [ ]:
!cp -r checkpoint.zip /content/drive/MyDrive/

### Тестирование

In [22]:
!unzip /content/drive/MyDrive/checkpoint.zip

Archive:  /content/drive/MyDrive/checkpoint.zip
   creating: content/bert-sentiment-split/checkpoint-6630/
  inflating: content/bert-sentiment-split/checkpoint-6630/scheduler.pt  
  inflating: content/bert-sentiment-split/checkpoint-6630/tokenizer.json  
  inflating: content/bert-sentiment-split/checkpoint-6630/tokenizer_config.json  
  inflating: content/bert-sentiment-split/checkpoint-6630/vocab.txt  
  inflating: content/bert-sentiment-split/checkpoint-6630/rng_state.pth  
  inflating: content/bert-sentiment-split/checkpoint-6630/optimizer.pt  
  inflating: content/bert-sentiment-split/checkpoint-6630/trainer_state.json  
  inflating: content/bert-sentiment-split/checkpoint-6630/special_tokens_map.json  
  inflating: content/bert-sentiment-split/checkpoint-6630/config.json  
  inflating: content/bert-sentiment-split/checkpoint-6630/training_args.bin  
  inflating: content/bert-sentiment-split/checkpoint-6630/model.safetensors  


In [23]:
# checkpoint_path = "/content/bert-sentiment-split/checkpoint-6630"
checkpoint_path = "/content/content/bert-sentiment-split/checkpoint-6630"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

In [25]:
def test(dataset_, labels_, n):
  raw_pred = trainer.predict(dataset_)
  preds = raw_pred.predictions.reshape(-1)
  labels = np.array(labels_)

  mae = mean_absolute_error(labels, preds)

  print(f'MAE: {mae:.4}')

  print("Результаты на тестовой выборке:")
  print("Предсказание\tИстинная метка")

  for p, l in list(zip(preds, labels))[:n]:
      print(f'{p:.2f}\t\t{l:.2f}')

In [27]:
test(dataset["test"], test_labels, 15)

MAE: 1.268
Результаты на тестовой выборке:
Предсказание	Истинная метка
8.57		9.00
2.44		2.00
9.11		10.00
8.80		8.00
2.78		3.00
2.71		3.00
8.03		3.00
1.36		1.00
1.64		4.00
5.09		3.00
1.65		2.00
1.71		7.00
1.90		1.00
9.40		9.00
5.21		4.00


## Собственные примеры

In [28]:
sentences = """
I am absolutely delighted with watching it!
Hi! I'm sick and won't be coming to class.
I am crying. I am sad. Nobody love me.
I'm so glad I caught the train!
How wonderful, i got cold and died.
"""


grade = [10., 4., 2., 8., 3.]

In [29]:
sentences_list = [s.strip() for s in sentences.split('\n') if s.strip()]
sentences_list

['I am absolutely delighted with watching it!',
 "Hi! I'm sick and won't be coming to class.",
 'I am crying. I am sad. Nobody love me.',
 "I'm so glad I caught the train!",
 'How wonderful, i got cold and died.']

In [30]:
sentences_dataset = Dataset.from_dict({
      "text": sentences_list,
      "label": grade
})

sentences_dataset = sentences_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [31]:
test(sentences_dataset, grade, 5)

MAE: 1.332
Результаты на тестовой выборке:
Предсказание	Истинная метка
9.83		10.00
3.41		4.00
1.53		2.00
9.85		8.00
6.58		3.00
